In [10]:
import ipywidgets as widgets
from IPython.display import display
import time
import threading
import os
from pathlib import Path


class PNGSequencePlayer:
    def __init__(self, file_paths, fps=24):
        """
        file_paths: A list of string paths to the .png files (sorted).
        fps: Default playback speed.
        """
        # 1. Pre-load all PNG bytes into memory for zero-latency playback
        self.frames_data = []
        try:
            for p in file_paths:
                with open(p, 'rb') as f:
                    self.frames_data.append(f.read())
        except Exception as e:
            print(f"Error loading files: {e}")
            return

        self.n_frames = len(self.frames_data)
        self.is_paused_event = False

        # --- UI Components ---

        # We tell the widget these are PNGs
        self.image_widget = widgets.Image(
            value=self.frames_data[0],
            format='png',
            layout=widgets.Layout(width='auto', max_width='250px')
        )

        # Animation Controller
        self.play_widget = widgets.Play(
            value=0, min=0, max=self.n_frames - 1,
            step=1, interval=int(1000/fps),
            description="Press play", repeat=False
        )

        # Scrubber
        self.slider = widgets.IntSlider(
            value=0, min=0, max=self.n_frames - 1, description="Frame"
        )

        # Controls
        self.fps_input = widgets.BoundedIntText(
            value=fps, min=1, max=120, step=1, description='FPS:',
            layout=widgets.Layout(width='140px')
        )

        self.loop_box = widgets.Checkbox(value=False, description="Loop")

        # Pause Logic UI
        self.pause_enable = widgets.Checkbox(value=True, description="Wait 2s @ Frame:")
        self.pause_idx = widgets.BoundedIntText(
            value=12, min=0, max=self.n_frames-1, description='',
            layout=widgets.Layout(width='60px')
        )

        # --- Logic Wiring ---

        widgets.jslink((self.play_widget, 'value'), (self.slider, 'value'))

        self.slider.observe(self.on_frame_change, names='value')
        self.fps_input.observe(self.update_speed, names='value')
        self.loop_box.observe(self.update_loop, names='value')

        # --- Layout ---

        # Group the specific pause controls
        pause_group = widgets.HBox([self.pause_enable, self.pause_idx])

        controls = widgets.VBox([
            widgets.HBox([self.play_widget, self.slider]),
            widgets.HBox([self.fps_input, self.loop_box, pause_group])
        ])

        self.ui = widgets.VBox([self.image_widget, controls])

    def on_frame_change(self, change):
        frame_idx = change['new']

        # DIRECT BYTE SWAP - Extremely fast
        self.image_widget.value = self.frames_data[frame_idx]

        # Check for "Magic Pause"
        if (self.play_widget.playing and
            self.pause_enable.value and
            frame_idx == self.pause_idx.value and
            not self.is_paused_event):

            self.trigger_pause()

    def trigger_pause(self):
        """Stops animation, waits 2s in background thread, resumes."""
        self.is_paused_event = True
        self.play_widget.playing = False # Stop

        def resume_worker():
            time.sleep(2)
            self.play_widget.playing = True # Resume
            # Tiny buffer to ensure we don't re-trigger on the same millisecond
            time.sleep(0.2)
            self.is_paused_event = False

        threading.Thread(target=resume_worker).start()

    def update_speed(self, change):
        if change['new'] > 0:
            self.play_widget.interval = int(1000 / change['new'])

    def update_loop(self, change):
        self.play_widget.repeat = change['new']

    def show(self):
        display(self.ui)

In [11]:
a = PNGSequencePlayer(sorted(Path('./1s/').glob('*.png')))
a.show()


In [ ]:
dir(a)

In [ ]:
!pip install ipywidgets

In [ ]:
!pip install imageio

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import imageio.v3 as iio

def play_video_with_speed(path, default_speed=50):
    # 1. Read the video/gif frames
    # imageio handles mpeg, gif, mp4, etc.
    try:
        frames = iio.imread(path)
    except Exception as e:
        print(f"Error loading video: {e}")
        return

    # 2. Create the Image Widget
    # We convert the first frame to bytes to initialize
    img_widget = widgets.Image(
        value=iio.imwrite("<bytes>", frames[0], extension=".png"),
        format='png',
        width=500
    )

    # 3. Create the Player Widget (Logic)
    # interval = milliseconds between frames
    play = widgets.Play(
        value=0,
        min=0,
        max=len(frames) - 1,
        step=1,
        interval=default_speed,
        description="Press play",
    )

    # 4. Create a Speed Slider
    speed_slider = widgets.IntSlider(
        value=default_speed,
        min=10,
        max=500,
        step=10,
        description='Speed (ms):'
    )

    # 5. Link the widgets
    # When the 'play' widget advances, update the image
    def on_frame_change(change):
        frame_index = change['new']
        # Convert numpy frame to png bytes for the browser
        img_widget.value = iio.imwrite("<bytes>", frames[frame_index], extension=".png")

    play.observe(on_frame_change, names='value')

    # Link speed slider to the play interval
    widgets.jslink((play, 'interval'), (speed_slider, 'value'))

    # Optional: A slider to scrub through frames manually
    scrub_slider = widgets.IntSlider(min=0, max=len(frames)-1)
    widgets.jslink((play, 'value'), (scrub_slider, 'value'))

    # 6. Display
    display(widgets.VBox([img_widget,  widgets.HBox([play, scrub_slider]), speed_slider]))

# Usage
# play_video_with_speed('my_animation.gif')
# play_video_with_speed('old_video.mpg')

play_video_with_speed('1s/1s.gif')
